Read in dataset and set curpath to working directory

In [23]:
import sys
import os
import inspect
import numpy as np
import pandas as pd
from scipy.stats import skew
from scipy.stats import kurtosis

parentPath = '/'.join(sys.path[0].split('/')[:-1])

Create Spark Context & SQLContext

In [24]:
# Creating Spark Context
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession

# Run the first time:
sc = SparkContext("local")

# Use to rerun script:
# sc = SparkContext.getOrCreate("local")
sqlContext = SQLContext(sc)
spark = SparkSession.builder.getOrCreate()

Load dataset from csv, using the "Expedia Hotel Recommendations" Kaggle dataset: https://www.kaggle.com/c/expedia-hotel-recommendations/overview

In [26]:
dfExpedia = spark.read.load(
  parentPath+'/data/train.csv',
  format="csv",
  sep=",",
  inferSchema=True,
  header=True
)

# Expose as SQL table
dfExpedia.createOrReplaceTempView('dfExpedia')

How many records in the training dataset?

In [27]:
dfExpedia.count()

37670293

How many columns in the training dataset?

In [28]:
len(dfExpedia.columns)

24

Take a 1% random sample of the training dataset

In [29]:
dfExpediaSample = dfExpedia.sample(withReplacement=False,fraction=0.01,seed=8)

How many records are in this sample?

In [30]:
dfExpediaSample.count()

375534

Preview the schema inferred when the dataset was read in

In [31]:
dfExpediaSample.printSchema()

root
 |-- date_time: timestamp (nullable = true)
 |-- site_name: integer (nullable = true)
 |-- posa_continent: integer (nullable = true)
 |-- user_location_country: integer (nullable = true)
 |-- user_location_region: integer (nullable = true)
 |-- user_location_city: integer (nullable = true)
 |-- orig_destination_distance: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- is_mobile: integer (nullable = true)
 |-- is_package: integer (nullable = true)
 |-- channel: integer (nullable = true)
 |-- srch_ci: timestamp (nullable = true)
 |-- srch_co: timestamp (nullable = true)
 |-- srch_adults_cnt: integer (nullable = true)
 |-- srch_children_cnt: integer (nullable = true)
 |-- srch_rm_cnt: integer (nullable = true)
 |-- srch_destination_id: integer (nullable = true)
 |-- srch_destination_type_id: integer (nullable = true)
 |-- is_booking: integer (nullable = true)
 |-- cnt: integer (nullable = true)
 |-- hotel_continent: integer (nullable = true)
 |-- hotel_country: 

Parse out timestamp and unique identifier columns

In [ ]:
dfExpediaSample = dfExpediaSample.selectExpr(
                                             'site_name'
                                            ,'posa_continent'
                                            ,'user_location_country'
                                            ,'user_location_region'
                                            ,'user_location_city'
                                            ,'orig_destination_distance'
                                            ,'is_mobile'
                                            ,'is_package'
                                            ,'srch_adults_cnt'
                                            ,'srch_children_cnt'
                                            ,'srch_rm_cnt'
                                            ,'srch_destination_id'
                                            ,'srch_destination_type_id'
                                            ,'is_booking'
                                            ,'cnt'
                                            ,'hotel_continent'
                                            ,'hotel_country'
                                            ,'hotel_market'
                                            ,'hotel_cluster'
                                            )

dfExpediaSample.createOrReplaceTempView('dfExpediaSample') # Expose as SQL table

Convert Spark data.frame to Pandas data.frame

In [ ]:
pd_dfExpediaSample = dfExpediaSample.toPandas()

Write sample data.frame to disk

In [ ]:
pd_dfExpediaSample.to_csv(parentPath+'/data/pd___dfExpediaSample.csv', index=False)

In [ ]:
pd_dfExpediaSample.head()